In [1]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_capability()

(7, 5)

In [2]:
from vllm import LLM
import torch
from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import os
os.environ['VLLM_USE_MODELSCOPE'] = "True"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
model_name = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(model_name, trust_remote_code=True,dtype=torch.float16,tensor_parallel_size=2)

2024-04-07 20:57:55,634 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-07 20:57:55,637 - modelscope - INFO - Loading ast index from /home/userroot/.cache/modelscope/ast_indexer
2024-04-07 20:57:55,685 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 821edacd9c7695756b5e5583d4438f12 and a total number of 972 components indexed
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 04-07 20:57:57 config.py:767] Casting torch.bfloat16 to torch.float16.


2024-04-07 20:58:00,216	INFO worker.py:1752 -- Started a local Ray instance.


INFO 04-07 20:58:01 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='/home/userroot/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat', tokenizer='/home/userroot/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir='/home/userroot/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat', load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-07 20:58:08 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-07 20:58:08 selector.py:25] Using XFormers backend.
(RayWorkerVllm pid=667006) INFO 04-07 20:58:10 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
(RayWorkerVllm pid=667006) INFO 04-07 20:58:10 selector.py:25] Using XFormers backend.
INFO 04-07 20:58:10 pynccl_utils.py:45] vLLM is using nccl==2.18.1
(RayWorkerVllm pid=667006) INFO 04-07 20:58:10 pynccl_utils.py:45] vLLM is using nccl==2.18.1
INFO 04-07 20:58:25 model_runner.py:104] Loading model weights took 6.4663 GB
(RayWorkerVllm pid=667006) INFO 04-07 20:58:29 model_runner.py:104] Loading model weights took 6.4663 GB
INFO 04-07 20:58:31 ray_gpu_executor.py:240] # GPU blocks: 3255, # CPU blocks: 1092
INFO 04-07 20:58:35 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' 

In [6]:
from vllm import LLM, SamplingParams
import pandas as pd
import json
data_path = "/home/userroot/dev/iTransformer/data/iTransformer_datasets/test_stock_news.jsonl"
df_news = pd.read_json(data_path,lines=True)

with open('index_300_keynotes.jsonl','w',encoding='utf-8') as f:
    for i , line in df_news.iterrows():
        news = line['content'][0:4000]
        messages = [
                    

                    {"role": "user", "content": f"从下面内容中总结内容\n{news}"},
                    {"role":"system", "content":f"你是一名专业的投研分析人员，你的任务是进行投研分析,先总结，然后发布评论，如果无法总结，请回答不知道。评论内容在总结内容之后，以【评论】开头。全部内容不要超过150字,不允许分段。"},

                ]

        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, return_tensors="pt",truncation='only_first')
        sampling_params = SamplingParams(temperature=0, top_p=0.95, max_tokens=300)
        
        line['key_note'] = llm.generate(prompt,sampling_params)[0].outputs[0].text
        json.dump(line.to_json(), f, ensure_ascii=False)

f.close()

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 